# 🚀 Professionally You Project

## 📲 Tool Setup: Pushover  

Pushover helps you send instant push notifications straight to your phone. Setup takes just a few minutes:  

---

### 🛠 Steps

1. **Sign Up**  
   Go to [pushover.net](https://pushover.net/), click **Login/Signup**, and create an account.  

2. **Create an App Token**  
   - On the dashboard, click **“Create an Application/API Token”**  
   - Name it (e.g., *Agents*) → then click **Create Application**.  

3. **Update `.env` File**  

   ```bash
   PUSHOVER_USER=<your user key from top-right of home screen>
   PUSHOVER_TOKEN=<your app token from the new app>


In [1]:
# imports

from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

In [2]:
# The usual start

load_dotenv(override=True)
openai = OpenAI()

In [4]:
# For pushover

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

if pushover_user:
    print(f"Pushover user found and starts with {pushover_user[0]}")
else:
    print("Pushover user not found")

if pushover_token:
    print(f"Pushover token found and starts with {pushover_token[0]}")
else:
    print("Pushover token not found")

Pushover user found and starts with u
Pushover token found and starts with a


In [5]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [8]:
push("HEY!!")

Push: HEY!!


In [9]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

In [10]:
def record_unknown_question(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

In [11]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "required": ["name"],
        "additionalProperties": False
    }
}

In [12]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [13]:
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

In [11]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'Use this tool to record that a user is interested in being in touch and provided an email address',
   'parameters': {'type': 'object',
    'properties': {'email': {'type': 'string',
      'description': 'The email address of this user'},
     'name': {'type': 'string',
      'description': "The user's name, if they provided it"},
     'notes': {'type': 'string',
      'description': "Any additional information about the conversation that's worth recording to give context"}},
    'required': ['name'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': "The question that couldn't be answered"}},
    'required': ['questi

In [14]:
# This function can take a list of tool calls, and run them. This is the IF statement!!

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        # THE BIG IF STATEMENT!!!

        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)

        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results


In [15]:
globals()["record_unknown_question"]("this is a really hard question")

Push: Recording this is a really hard question asked that I couldn't answer


{'recorded': 'ok'}

In [16]:
# This is a more elegant way that avoids the IF statement.

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [17]:
reader = PdfReader("personal_info/Profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("personal_info/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Ezekiel Yohanna"

In [18]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. Also ensure that the email is in the correct format"

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [19]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:

        # This is the call to the LLM - see that we pass in the tools json

        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)

        finish_reason = response.choices[0].finish_reason
        
        # If the LLM wants to call a tool, we do that!
         
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

In [20]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Ah, got it! You want **just the deployment guide** formatted nicely for a notebook. Here’s a clean, polished version you can paste directly into a **Markdown cell**:

````markdown
# 🚀 Deployment Guide  

We’ll deploy the app (`app.py`) to **Hugging Face Spaces**.  

> ⚡ **Before you start:**  
> - Update the files in the **`me`** directory (LinkedIn profile + `summary.txt`) to reflect your info.  
> - Update `self.name = "Ezekiel"` in `app.py`.  
> - Ensure there’s no `README.md` in `1_foundations` (delete it if it exists).  

---

## 🛠 Steps to Deploy  

1. **Set Up Hugging Face**  
   - Go to [huggingface.co](https://huggingface.co) → create an account.  
   - Avatar menu → **Access Tokens** → **Create New Token** (give it **WRITE** permissions).  

2. **Install CLI Tool**  
   In your terminal:  
   ```bash
   uv tool install "huggingface_hub[cli]"
   hf auth login
   hf auth whoami
````

💡 Use your HF token during login.

3. **Update `.env` File**

   ```bash
   HF_TOKEN=hf_xxx
   ```

4. **Deploy**
   From the `1_foundations` folder:

   ```bash
   uv run gradio deploy
   ```

   Follow prompts:

   * Space name: `career_conversation`
   * Entry point: `app.py`
   * Hardware: `cpu-basic`
   * Secrets: Yes → add your keys (`OPENAI_API_KEY`, `PUSHOVER_USER`, `PUSHOVER_TOKEN`)
   * GitHub Actions: No

---

## 🔑 About Secrets

* When prompted, enter **key name** (e.g., `OPENAI_API_KEY`) then **value** (e.g., `sk-proj-...`).
* Alternatively:

  * Log into [huggingface.co](https://huggingface.co)
  * Avatar → Profile → Select your Space → **⚙ Settings** → **Variables and Secrets**

---

## ✅ Done!

Your app should now be live 🎉

👉 Example: [Career Conversation Space](https://huggingface.co/spaces/ezzybala/career_conversation)

---

## 🔄 Redeploying or Resetting

* To redeploy, you may need to **delete the auto-created `README.md`** inside the `career_conversation` folder.
* To delete a Space:

  1. Log into Hugging Face
  2. Avatar → Profile → Select the Space → ⚙ Settings
  3. Scroll down → Delete

📖 More info: [Gradio: Hosting on HF Spaces](https://www.gradio.app/guides/sharing-your-app#hosting-on-hf-spaces)



<table style="width:100%; background-color:#f0f8ff; border-radius:10px; padding:10px; border-collapse: separate; border-spacing:10px;">
    <tr>
        <td style="width:160px; text-align:center; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="border-radius:10px;" />
        </td>
        <td style="vertical-align: middle;">
            <h2 style="margin:0; color:#0077cc; font-family: 'Segoe UI', sans-serif;">Commercial Implications</h2>
            <p style="margin:5px 0 0 0; color:#005f99; font-family: 'Segoe UI', sans-serif; font-size:14px; line-height:1.5;">
                Beyond personal use (your career alter-ego), this has <strong>real business applications</strong> wherever you need an AI assistant with domain expertise and the ability to interact with the real world.
            </p>
        </td>
    </tr>
</table>
